<a href="https://colab.research.google.com/github/gfelekis/MSc-Dissertation/blob/master/MCD_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Imports
!pip install GPy

import numpy as np
import scipy as sp
import scipy.stats
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import zipfile
import urllib.request
import os
import GPy
import time
import copy
import math
import tqdm
from tqdm import tqdm
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import Optimizer
from torch.optim.sgd import SGD
from sklearn.model_selection import KFold

import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from tqdm import tqdm, trange
from google.colab import files
%config InlineBackend.figure_format = 'svg'

In [ ]:
#@title UCI Datasets
#Boston housing dataset
np.random.seed(2)
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data" --no-check-certificate 
data1 = pd.read_csv('housing.data', header=0, delimiter="\s+").values
data1 = data1[np.random.permutation(np.arange(len(data1)))]

# Concrete compressive dataset
np.random.seed(2)
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/compressive/Concrete_Data.xls" --no-check-certificate
data2 = pd.read_excel('Concrete_Data.xls', header=0, delimiter="\s+").values
data2 = data2[np.random.permutation(np.arange(len(data2)))]

# Energy efficiency dataset
np.random.seed(2)
!wget "http://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx" --no-check-certificate
data3 = pd.read_excel('ENB2012_data.xlsx', header=0, delimiter="\s+").values
data3 = data3[np.random.permutation(np.arange(len(data3)))]

# Red wine dataset
np.random.seed(2)
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv" --no-check-certificate 
data4 = pd.read_csv('winequality-red.csv', header=1, delimiter=';').values
data4 = data4[np.random.permutation(np.arange(len(data4)))]

#Yacht dataset
np.random.seed(2)
!wget "http://archive.ics.uci.edu/ml/machine-learning-databases/00243/yacht_hydrodynamics.data" --no-check-certificate 
data5 = pd.read_csv('yacht_hydrodynamics.data', header=1, delimiter='\s+').values
data5 = data5[np.random.permutation(np.arange(len(data5)))]

In [ ]:
torch.cuda.manual_seed_all(999)

In [ ]:
def to_variable(var=(), cuda=True, volatile=False):
    out = []
    for v in var:
        
        if isinstance(v, np.ndarray):
            v = torch.from_numpy(v).type(torch.FloatTensor)

        if not v.is_cuda and cuda:
            v = v.cuda()

        if not isinstance(v, Variable):
            v = Variable(v, volatile=volatile)

        out.append(v)
    return out

In [ ]:
def log_gaussian_loss(output, target, sigma, no_dim):
    exponent = -0.5*(target - output)**2/sigma**2
    log_coeff = -no_dim*torch.log(sigma) - 0.5*no_dim*np.log(2*np.pi)
    
    return - (log_coeff + exponent).sum()

In [ ]:
class MC_Dropout_Model_UCI(nn.Module):
    def __init__(self, input_dim, output_dim, num_units, drop_prob):
        super(MC_Dropout_Model_UCI, self).__init__()
        #torch.manual_seed(42)
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.drop_prob = drop_prob
        self.num_units = num_units

        # network with two hidden and one output layer
        if len(num_units) == 1:
          self.layer1 = nn.Linear(input_dim, num_units[0])
          self.layer2 = nn.Linear(num_units[0], 2*output_dim)
        if len(num_units) == 2:
          self.layer1 = nn.Linear(input_dim, num_units[0])
          self.layer2 = nn.Linear(num_units[0], num_units[1])
          self.layer3 = nn.Linear(num_units[1], 2*output_dim)
        elif len(num_units) == 3:
          self.layer1 = nn.Linear(input_dim, num_units[0])
          self.layer2 = nn.Linear(num_units[0], num_units[1])
          self.layer3 = nn.Linear(num_units[1], num_units[2])
          self.layer4 = nn.Linear(num_units[2], 2*output_dim)

        self.activation = nn.ReLU(inplace = True)

    
    def forward(self, x):
        x = x.view(-1, self.input_dim)

        if len(self.num_units) == 1:
          x = self.layer1(x)
          x = self.activation(x)
          x = F.dropout(x, p=self.drop_prob, training=True)
          x = self.layer2(x)

        if len(self.num_units) == 2:
          x = self.layer1(x)
          x = self.activation(x)
          x = F.dropout(x, p=self.drop_prob, training=True)

          x = self.layer2(x)
          x = self.activation(x)
          x = F.dropout(x, p=self.drop_prob, training=True)

          x = self.layer3(x)

        elif len(self.num_units) == 3:
          x = self.layer1(x)
          x = self.activation(x)
          x = F.dropout(x, p=self.drop_prob, training=True)

          x = self.layer2(x)
          x = self.activation(x)
          x = F.dropout(x, p=self.drop_prob, training=True)

          x = self.layer3(x)
          x = self.activation(x)
          x = F.dropout(x, p=self.drop_prob, training=True)

          x = self.layer4(x)
        
        return x


def train_mc_dropout(data, drop_prob, n_splits, num_epochs, num_units, batch_size, learn_rate, weight_decay, log_every, num_samples):
    #torch.manual_seed(42)
    kf = KFold(n_splits=n_splits)
    in_dim = data.shape[1] - 1
    train_logliks, test_logliks = [], []
    train_rmses, test_rmses = [], []

    for j, idx in enumerate(kf.split(data)):
        num = j+1
        print('SPLIT %d:' % num)
        train_index, test_index = idx

        x_train, y_train = data[train_index, :in_dim], data[train_index, in_dim:]
        x_test, y_test = data[test_index, :in_dim], data[test_index, in_dim:]

        x_means, x_stds = x_train.mean(axis = 0), x_train.var(axis = 0)**0.5
        y_means, y_stds = y_train.mean(axis = 0), y_train.var(axis = 0)**0.5

        x_train = (x_train - x_means)/x_stds
        y_train = (y_train - y_means)/y_stds

        x_test = (x_test - x_means)/x_stds
        y_test = (y_test - y_means)/y_stds

        net = MC_Dropout_Wrapper(network=MC_Dropout_Model_UCI(input_dim=in_dim, output_dim=1, num_units=num_units, drop_prob=drop_prob),
                                 learn_rate=learn_rate, batch_size=batch_size, weight_decay=weight_decay)

        fit_loss_train = np.zeros(num_epochs)

        for i in range(num_epochs):
            loss = net.fit(x_train, y_train)
                
            if i % log_every == 0 or i == num_epochs - 1:
                test_loss, rmse = net.get_loss_and_rmse(x_test, y_test, num_samples=num_samples)
                test_loss, rmse = test_loss.cpu().data.numpy(), rmse.cpu().data.numpy()

                print('Epoch: %4d, Train loss: %6.3f Test loss: %6.3f RMSE: %.3f' %
                      (i, loss.cpu().data.numpy()/len(x_train), test_loss/len(x_test), rmse*y_stds[0]))


        train_loss, train_rmse = net.get_loss_and_rmse(x_train, y_train, num_samples=num_samples)
        test_loss, test_rmse = net.get_loss_and_rmse(x_test, y_test, num_samples=num_samples)
        
        train_logliks.append((train_loss.cpu().data.numpy()/len(x_train) + np.log(y_stds)[0]))
        test_logliks.append((test_loss.cpu().data.numpy()/len(x_test) + np.log(y_stds)[0]))

        train_rmses.append(y_stds[0]*train_rmse.cpu().data.numpy())
        test_rmses.append(y_stds[0]*test_rmse.cpu().data.numpy())


    print('Train log. lik. = %6.3f +/- %6.3f' % (-np.array(train_logliks).mean(), np.array(train_logliks).var()**0.5))
    print('Test  log. lik. = %6.3f +/- %6.3f' % (-np.array(test_logliks).mean(), np.array(test_logliks).var()**0.5))
    print('Train RMSE      = %6.3f +/- %6.3f' % (np.array(train_rmses).mean(), np.array(train_rmses).var()**0.5))
    print('Test  RMSE      = %6.3f +/- %6.3f' % (np.array(test_rmses).mean(), np.array(test_rmses).var()**0.5))
    
    rmses =  list(np.array(test_rmses).flatten())
    print("Test LogLike for different folds: ", test_logliks)
    print("Test RMSEs   for different folds: ", rmses)

    metrics = {"train_log_like_mean": -np.array(train_logliks).mean(), "train_log_like_var": np.array(train_logliks).var()**0.5,
               "test_log_like_mean": -np.array(test_logliks).mean(), "test_log_like_var": np.array(test_logliks).var()**0.5,
               "train_rmse_mean": np.array(train_rmses).mean(), "train_rmse_var": np.array(train_rmses).var()**0.5,
               "test_rmse_mean": np.array(test_rmses).mean(), "test_rmse_var":np.array(test_rmses).var()**0.5,
               "rmse_values": list(np.array(test_rmses).flatten()),
               "loglik_values": list(np.array(test_logliks).flatten()),
               }

    return net, metrics

In [ ]:
class MC_Dropout_Wrapper:
    def __init__(self, network, learn_rate, batch_size , weight_decay):
        
        self.learn_rate = learn_rate
        self.batch_size = batch_size
        
        self.network = network
        self.network.cuda()
        
        self.optimizer = torch.optim.SGD(self.network.parameters(), lr=learn_rate, weight_decay=weight_decay)
        self.loss_func = log_gaussian_loss
    
    def fit(self, x, y):
        #torch.manual_seed(42)
        x, y = to_variable(var=(x, y), cuda=True)
        
        # reset gradient and total loss
        self.optimizer.zero_grad()
        
        output = self.network(x)
        loss = self.loss_func(output[:, :1], y, output[:, 1:].exp(), 1)
        
        loss.backward()
        self.optimizer.step()

        return loss
    
    def get_loss_and_rmse(self, x, y, num_samples):
        x, y = to_variable(var=(x, y), cuda=True)
        
        means, stds = [], []
        for i in range(num_samples):
            output = self.network(x)
            means.append(output[:, :1])
            stds.append(output[:, 1:].exp())
        
        means, stds = torch.cat(means, dim=1), torch.cat(stds, dim=1)
        mean = means.mean(dim=-1)[:, None]
        std = ((means.var(dim=-1) + stds.mean(dim=-1)**2)**0.5)[:, None]
        loss = self.loss_func(mean, y, std, 1)
        
        rmse = ((mean - y)**2).mean()**0.5

        return loss.detach().cpu(), rmse.detach().cpu()

## RUN THE EXPERIMENTS

# Regression on UCI data

In [ ]:
# set up the access to drive - we'll be saving our logs there
from google.colab import drive
drive.mount("/content/drive")
# define paths for the different experiments
mcd_path = "/content/drive/My Drive/"+"new_mcd_logs.txt"

In [ ]:
logs = []
### some params
n_splits = 30
n_epochs = 200
hidden   = 100
models, log_metrics = [], []

dataset = [data1, data2, data3, data4, data5] # list of all the datasets
dataset_names = ["Boston", "Concrete", "Energy", "Wine", "Yacht"]
hiddens = [1,2,3]

for i, data in enumerate(dataset):
  dataset_name = dataset_names[i]
  for h in hiddens:
    # run the training
    num_units = [hidden for i in range(h)]
    model, metric = train_mc_dropout(data=data, drop_prob=0.1, num_epochs=n_epochs, 
                                     n_splits=n_splits, num_units=num_units, batch_size =32, learn_rate=1e-4,
                                     weight_decay=1e-1/len(data)**0.5, num_samples=20, log_every=50)
    models.append(model)
    # record to file: 
    log = {"dataset": dataset_name,
           "loss": "mc_dropout",
           "alpha": "dropout_prob_"+str(0.1),
           "n_layers": h, 
           "constant_hidden_size": hidden, 
           "metrics": metric}
    logs.append(log)
    with open(mcd_path, "a") as f:
      f.write(str(log)+"\n")
      print(log)
    break

SPLIT 1:
Epoch:    0, Train loss:  1.419 Test loss:  1.184 RMSE: 5.419
Epoch:   50, Train loss:  0.429 Test loss:  0.252 RMSE: 2.472
Epoch:  100, Train loss:  0.264 Test loss:  0.183 RMSE: 2.596
Epoch:  150, Train loss:  0.296 Test loss:  0.222 RMSE: 2.322
Epoch:  199, Train loss:  0.199 Test loss:  0.445 RMSE: 3.932
SPLIT 2:
Epoch:    0, Train loss:  2.026 Test loss:  1.675 RMSE: 7.092
Epoch:   50, Train loss:  0.853 Test loss:  1.913 RMSE: 8.264
Epoch:  100, Train loss:  1.578 Test loss:  0.942 RMSE: 4.029
Epoch:  150, Train loss:  0.243 Test loss:  0.099 RMSE: 1.880
Epoch:  199, Train loss:  0.106 Test loss: -0.033 RMSE: 1.830
SPLIT 3:
Epoch:    0, Train loss:  1.308 Test loss:  1.083 RMSE: 5.191
Epoch:   50, Train loss:  1.360 Test loss:  1.233 RMSE: 7.896
Epoch:  100, Train loss:  0.360 Test loss:  0.412 RMSE: 3.163
Epoch:  150, Train loss:  0.224 Test loss:  0.017 RMSE: 2.106
Epoch:  199, Train loss:  0.171 Test loss:  0.052 RMSE: 2.261
SPLIT 4:
Epoch:    0, Train loss:  1.554 Te

# Regression on GP ground truth


In [ ]:
def plot_uncertainty_mcd(net, x_train, y_train):
  samples = []
  noises = []
  for i in range(1000):
      preds = net.network.forward(torch.linspace(-5, 5, 200).cuda()).cpu().data.numpy()
      samples.append(preds[:, 0])
      noises.append(np.exp(preds[:, 1]))
      
  samples = np.array(samples)
  noises = np.array(noises)
  means = (samples.mean(axis = 0)).reshape(-1)

  aleatoric = (noises**2).mean(axis = 0)**0.5
  epistemic = (samples.var(axis = 0)**0.5).reshape(-1)
  total_unc = (aleatoric**2 + epistemic**2)**0.5


  c = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
      '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

  plt.figure(figsize = (6, 5))
  plt.style.use('default')
  plt.scatter(x_train, y_train, s = 10, marker = 'x', color = 'black', alpha = 0.5)
  plt.fill_between(np.linspace(-5, 5, 200), means + aleatoric, means + total_unc, color = c[0], alpha = 0.3, label = 'Epistemic + Aleatoric')
  plt.fill_between(np.linspace(-5, 5, 200), means - total_unc, means - aleatoric, color = c[0], alpha = 0.3)
  plt.fill_between(np.linspace(-5, 5, 200), means - aleatoric, means + aleatoric, color = c[4], alpha = 0.4, label = 'Aleatoric')
  plt.plot(np.linspace(-5, 5, 200), means, color = 'black', linewidth = 1)
  plt.xlim([-5, 5])
  plt.ylim([-5, 7])
  plt.xlabel('$x$', fontsize=10)
  plt.title('MC dropout', fontsize=10)
  plt.tick_params(labelsize=10)
  plt.xticks(np.arange(-4, 5, 2))
  plt.yticks(np.arange(-4, 7, 2))
  plt.gca().set_yticklabels([])
  plt.gca().yaxis.grid(alpha=0.3)
  plt.gca().xaxis.grid(alpha=0.3)
  plt.savefig('mc_dropout_hetero.pdf', bbox_inches = 'tight')


  plt.show()

In [ ]:
def plot_uncertainty_3row_mcd(nets, x, y):
  fig, ax = plt.subplots(1, 3, figsize=(15, 4))
  fig.suptitle("MC Dropout h=1 | h=2 | h=3")
  for n, net in enumerate(nets):
    samples, noises = [], []
    for i in range(100):
        preds = nets[n].network.forward(torch.linspace(-5, 5, 200).cuda()).cpu().data.numpy()
        samples.append(preds[:, 0])
        noises.append(preds[:, 1])

    samples = np.array(samples)
    means = (samples.mean(axis = 0)).reshape(-1)
    noises = np.array(noises)

    aleatoric = (noises**2).mean(axis = 0)**0.5
    epistemic = samples.var(axis = 0)**0.5
    total_unc = (aleatoric**2 + epistemic**2)**0.5

    c = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
        '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

    ax[n].scatter(x_train, y_train, s = 10, marker = 'x', color = 'black', alpha = 0.5)
    ax[n].fill_between(np.linspace(-5, 5, 200), means + aleatoric, means + total_unc, color = c[0], alpha = 0.3, label = 'Epistemic + Aleatoric')
    ax[n].fill_between(np.linspace(-5, 5, 200), means - total_unc, means - aleatoric, color = c[0], alpha = 0.3)
    ax[n].fill_between(np.linspace(-5, 5, 200), means - aleatoric, means + aleatoric, color = c[4], alpha = 0.4, label = 'Aleatoric')
    ax[n].plot(np.linspace(-5, 5, 200), means, color = 'black', linewidth = 1)
    ax[n].set_xlim([-5, 5])
    ax[n].set_ylim([-5, 7])
    ax[n].set_xlabel('$x$', fontsize=10)
    ax[n].set_title("h = "+str(n+1), fontsize=10)
    ax[n].tick_params(labelsize=10)
    ax[n].set_xticks(np.arange(-4, 5, 2))
    ax[n].set_yticks(np.arange(-4, 7, 2))
    plt.gca().set_yticklabels([])
    ax[n].grid(alpha=0.3)

  plt.savefig('mcd_hetero.pdf', bbox_inches = 'tight')
  plt.show()

In [ ]:
np.random.seed(2)
no_points = 400
lengthscale = 1
variance = 1.0
sig_noise = 0.3
x = np.random.uniform(-3, 3, no_points)[:, None]
x.sort(axis = 0)


k = GPy.kern.RBF(input_dim=1, variance=variance)#, lengthscale=lengthscale)
C = k.K(x, x) + np.eye(no_points)*(x + 2)**2*sig_noise**2

y = np.random.multivariate_normal(np.zeros((no_points)), C)[:, None]
y = (y - y.mean())
x_train = x[75:325]
y_train = y[75:325]

num_epochs, batch_size = 2000, len(x_train)

#h_nets = []
for num_units in [[200], [200, 300], [200, 300, 200]]: 
  net = MC_Dropout_Wrapper(network=MC_Dropout_Model_UCI(input_dim=1, output_dim=1, num_units=num_units, drop_prob=0.5),
                          learn_rate=1e-4, batch_size=batch_size, weight_decay=1e-2)

  fit_loss_train = np.zeros(num_epochs)
  best_net, best_loss = None, float('inf')
  nets, losses = [], []

  for i in range(num_epochs): # 
      
      loss = net.fit(x_train, y_train)
      
      if i % 100 == 0:
          print('Epoch: %4d, Train loss = %7.3f' % (i, loss.cpu().data.numpy()/batch_size))
  #h_nets.append(copy.copy(net))

  plot_uncertainty_mcd(net, x_train, y_train)